## Read Data

In [1]:
import pandas as pd
train=pd.read_csv("/Users/deepak/Downloads/data_science_challenge/train.csv")

## Description of the features
http://www.nyc.gov/html/nypd/downloads/ptrain/analysis_and_planning/NYPDIncidentLevelDataFootnotes.ptrain

Part 1: Build a model that can predict the type of ‘Offense’ given all the other features

Use the train .csv file to train a model to predict the ‘Offense’ column.

Your returned assignment should include:

* Working code which solves the problem, along with any code used for EDA, etc
* Classification accuracy of a dummy model that always guesses “GRAND LARCENY”.
* Classification accuracy of your trained model evaluated on the train data ([*]train.csv).

If you wish to include other metrics you are welcome to do so, but please include the accuracy (defined as the fraction of offenses predicted correctly).

## HEAD

In [338]:
train.head(3)

,Identifier,Occurrence Datetime,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1,Occurrence Date
0,4eaf2b62,02/13/2013 12:00:00 AM,Wednesday,Feb,13,2013,0,2,14,2013,GRAND LARCENY,H,13,MANHATTAN,N.Y. POLICE DEPT,985716,209911,"(40.7428419120001, -73.9947109889999)",2013-02-13
1,cacec67c,02/13/2013 12:00:00 AM,Wednesday,Feb,13,2013,0,5,20,2013,GRAND LARCENY,I,52,BRONX,N.Y. POLICE DEPT,1016552,260706,"(40.88220104, -73.88318653)",2013-02-13
2,ca4bc93e,02/13/2013 12:01:00 AM,Wednesday,Feb,13,2013,0,2,13,2013,FELONY ASSAULT,A,100,QUEENS,N.Y. POLICE DEPT,1041165,155066,"(40.592122008, -73.795071599)",2013-02-13


## FIRST THOUGHTS:

 CHALLENGES:
* Many categorical variables and categorical output
* Categorical values must be encoded
* Time series data
* DATETIME module could be used to get timestamp from Occurrence Datetime.
* Features that dont provide value/ features that repeat- Identifier, Occurrence Datetime, Occurrence Date
* Possible collinearity between Actual event time and Reported event time.
* Zipcode can be obtained from lat-long using google maps
* latlong can be used to visualize on top of a map the distribution of crime
* Look for seasonal effects
* Not sure how to use xcoord, ycoord.

## SHAPE AND SIZE

In [339]:
print("SHAPE: ",train.shape)
print("SIZE:",train.size)

SHAPE:  (213689, 19)
SIZE: 4060091


In [340]:
train.dtypes

Identifier             object
Occurrence Datetime    object
Day of Week            object
Occurrence Month       object
Occurrence Day          int64
Occurrence Year         int64
Occurrence Hour         int64
CompStat Month          int64
CompStat Day            int64
CompStat Year           int64
Offense                object
Sector                 object
Precinct                int64
Borough                object
Jurisdiction           object
XCoordinate             int64
YCoordinate             int64
Location 1             object
Occurrence Date        object
dtype: object

In [341]:
print(train.describe())

       Occurrence Day  Occurrence Year  Occurrence Hour  CompStat Month  \
count   213689.000000    213689.000000    213689.000000   213689.000000   
mean        15.705778      2013.487891        12.776156        6.711941   
std          8.824787         0.499855         6.753176        3.386492   
min          1.000000      2013.000000         0.000000        1.000000   
25%          8.000000      2013.000000         8.000000        4.000000   
50%         16.000000      2013.000000        14.000000        7.000000   
75%         23.000000      2014.000000        18.000000       10.000000   
max         31.000000      2014.000000        23.000000       12.000000   

        CompStat Day  CompStat Year       Precinct   XCoordinate  \
count  213689.000000  213689.000000  213689.000000  2.136890e+05   
mean       15.797477    2013.510256      63.444520  1.005701e+06   
std         8.775506       0.522588      34.490573  2.058782e+04   
min         1.000000    2013.000000       1.000000  

In [342]:
train.skew()

Occurrence Day     0.002951
Occurrence Year    0.048449
Occurrence Hour   -0.370107
CompStat Month    -0.099611
CompStat Day      -0.000057
CompStat Year      0.202945
Precinct          -0.028657
XCoordinate       -0.020874
YCoordinate        0.092971
dtype: float64

In [382]:
offense= (train['Occurrence Datetime'].groupby(train['Offense']).count())

import plotly.plotly as py
import plotly.graph_objs as go

data = [go.Bar(
            x=list(offense.index),
            y=list(offense)
    )]

layout = go.Layout(
    yaxis=dict(title='Count'),
    xaxis=dict(title='Offense'),
    title='Distribution of Offenses')

fig=go.Figure(data=data, layout=layout)
py.iplot(fig)

Grand larceny is the most common crime and Murder/Rape are the least

In [358]:
(train['Occurrence Datetime'].groupby(train['Occurrence Year']).count())

Occurrence Year
2013    109432
2014    104257
Name: Occurrence Datetime, dtype: int64

In [387]:
dom=train["Occurrence Month"].value_counts()

dom = [go.Bar(
            x=list(dom.index),
            y=list(dom)
    )]


layout = go.Layout(
    yaxis=dict(title='Count'),
    xaxis=dict(title='Day of Month'),
    title='Offense by Month')    
    
fig3=go.Figure(data=dom, layout=layout)

py.iplot(fig3, filename='basic-bar')

Offense count seems to be maximum in summer- July and August (31 days), minimum on Feb(28 days)

In [383]:
dow=train["Day of Week"].value_counts()

dow = [go.Bar(
            x=list(dow.index),
            y=list(dow)
    )]


layout = go.Layout(
    yaxis=dict(title='Count'),
    xaxis=dict(title='Day of Week'),
    title='Offense by Day of week')    
    
fig2=go.Figure(data=dow, layout=layout)

py.iplot(fig2, filename='basic-bar')

More offenses occur on Weekends(Friday and Saturday) which is expected and least on Sunday (Surprising).

In [390]:
hod=train["Occurrence Hour"].value_counts()

hod = [go.Bar(
            x=list(hod.index),
            y=list(hod)
    )]


layout = go.Layout(
    yaxis=dict(title='Count'),
    xaxis=dict(title='Hour of Day'),
    title='Offense by Hour of Day')    
    
fig4=go.Figure(data=hod, layout=layout)

py.iplot(fig4, filename='basic-bar')

Number of offense seems to be maximum between 3 PM and 7 PM and minimum between 4 AM and 7 AM

In [354]:
train.head()

,Identifier,Occurrence Datetime,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1,Occurrence Date
0,4eaf2b62,02/13/2013 12:00:00 AM,Wednesday,Feb,13,2013,0,2,14,2013,GRAND LARCENY,H,13,MANHATTAN,N.Y. POLICE DEPT,985716,209911,"(40.7428419120001, -73.9947109889999)",2013-02-13
1,cacec67c,02/13/2013 12:00:00 AM,Wednesday,Feb,13,2013,0,5,20,2013,GRAND LARCENY,I,52,BRONX,N.Y. POLICE DEPT,1016552,260706,"(40.88220104, -73.88318653)",2013-02-13
2,ca4bc93e,02/13/2013 12:01:00 AM,Wednesday,Feb,13,2013,0,2,13,2013,FELONY ASSAULT,A,100,QUEENS,N.Y. POLICE DEPT,1041165,155066,"(40.592122008, -73.795071599)",2013-02-13
3,19f9636c,02/13/2013 12:05:00 AM,Wednesday,Feb,13,2013,0,2,13,2013,FELONY ASSAULT,H,62,BROOKLYN,N.Y. POLICE DEPT,984298,163775,"(40.61620917, -73.999828549)",2013-02-13
4,629302ce,02/13/2013 12:05:00 AM,Wednesday,Feb,13,2013,0,2,13,2013,FELONY ASSAULT,D,52,BRONX,N.Y. POLICE DEPT,1013228,253750,"(40.863120505, -73.8952373069999)",2013-02-13


## Classification accuracy of a dummy model that always guesses “GRAND LARCENY”.

In [449]:
test=pd.read_csv("/Users/deepak/Downloads/data_science_challenge/test.csv")


In [450]:
print("ON TRAINING DATA",train[train.Offense=="GRAND LARCENY"].shape[0]/df.shape[0]*100)
print("ON TEST DATA,test[test.Offense=="GRAND LARCENY"].shape[0]/df.shape[0]*100)

40.83551329268236
19.094572018213384


## DROP COLUMNS

In [40]:
def drop_colums(data):
    columns = ["Identifier","Occurrence Datetime","Occurrence Date","XCoordinate","YCoordinate","Location 1"]
    data=data.drop(columns,axis=1)
    return data

In [5]:
train_data=drop_colums(train)
train_data.head()

,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Sector,Precinct,Borough,Jurisdiction
0,Wednesday,Feb,13,2013,0,2,14,2013,GRAND LARCENY,H,13,MANHATTAN,N.Y. POLICE DEPT
1,Wednesday,Feb,13,2013,0,5,20,2013,GRAND LARCENY,I,52,BRONX,N.Y. POLICE DEPT
2,Wednesday,Feb,13,2013,0,2,13,2013,FELONY ASSAULT,A,100,QUEENS,N.Y. POLICE DEPT
3,Wednesday,Feb,13,2013,0,2,13,2013,FELONY ASSAULT,H,62,BROOKLYN,N.Y. POLICE DEPT
4,Wednesday,Feb,13,2013,0,2,13,2013,FELONY ASSAULT,D,52,BRONX,N.Y. POLICE DEPT


### GET ZIPCPDE FROM LAT-LONG

In [284]:
import requests
import re

def zipcode_from_latlong(latitude,longitude):

#     latitude = 35.1330343
#     longitude = -90.0625056
    sensor = 'true'
    base = "http://maps.googleapis.com/maps/api/geocode/json?"
    params = "latlng={lat},{lon}&sensor={sen}".format(
        lat=latitude,
        lon=longitude,
        sen=sensor
    )
    url = "{base}{params}".format(base=base, params=params)
    response = requests.get(url)
    address = response.json()['results'][0]['formatted_address']
    
    
    ### Extract zipcode from address
    pattern=re.compile(r"\d{5}")
    matches=pattern.findall(address)
    
    return matches



['38103']

* Using Google maps API would take atleast 10 hrs to get  zipcode for all data.

In [296]:
zipcode_from_latlong(35.1330343,-90.0625056)

['38103']

In [ ]:
zipcodes=[]
for element in data["Location 1"]:
    try:

        lat=(float((element.split(",")[0]).replace("(","")))
        long=(float((element.split(",")[1]).replace(")","")))
        zipcodes.append(zipcode_from_latlong(lat,long))
    except IndexError:
        zipcodes.append("null")

In [330]:
len(zipcodes)  ## Extracted in 1 one hr

24778

In [454]:
zipcodes[1:5]

[['10467'], ['11692'], ['11204'], ['10458']]

## HANDLING 'OBJECT' DATATYPES

In [6]:
train_data.head(2)

,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Sector,Precinct,Borough,Jurisdiction
0,Wednesday,Feb,13,2013,0,2,14,2013,GRAND LARCENY,H,13,MANHATTAN,N.Y. POLICE DEPT
1,Wednesday,Feb,13,2013,0,5,20,2013,GRAND LARCENY,I,52,BRONX,N.Y. POLICE DEPT


In [7]:
obj_train = train_data.select_dtypes(include=['object']).copy()
obj_train.head()

,Day of Week,Occurrence Month,Offense,Sector,Borough,Jurisdiction
0,Wednesday,Feb,GRAND LARCENY,H,MANHATTAN,N.Y. POLICE DEPT
1,Wednesday,Feb,GRAND LARCENY,I,BRONX,N.Y. POLICE DEPT
2,Wednesday,Feb,FELONY ASSAULT,A,QUEENS,N.Y. POLICE DEPT
3,Wednesday,Feb,FELONY ASSAULT,H,BROOKLYN,N.Y. POLICE DEPT
4,Wednesday,Feb,FELONY ASSAULT,D,BRONX,N.Y. POLICE DEPT


## ENCODING

In [8]:
from sklearn.preprocessing import LabelEncoder

## obj_df = object data types dataframe
## data = entire dataframe
def encode(obj_df,data):
    """
    Encode object data types
    """
    for column in list(obj_df.columns):
             lb_make = LabelEncoder()
             obj_df[column] = lb_make.fit_transform(obj_df[column])
             data[column]=obj_df[column]
    return 0    

In [10]:
encode(obj_train,train_data)

0

In [12]:
train_data.head(3)

,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Sector,Precinct,Borough,Jurisdiction
0,6,3,13,2013,0,2,14,2013,2,8,13,3,7
1,6,3,13,2013,0,5,20,2013,2,9,52,1,7
2,6,3,13,2013,0,2,13,2013,1,1,100,4,7


## EMPTY VALUES?

In [462]:
train_data[train_data.isnull().any(axis=1)].head()

,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Sector,Precinct,Borough,Jurisdiction


In [260]:
#obj_train["sector"].value_counts()

## FIT and Predict

In [17]:
train_data.head()

,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Sector,Precinct,Borough,Jurisdiction
0,6,3,13,2013,0,2,14,2013,2,8,13,3,7
1,6,3,13,2013,0,5,20,2013,2,9,52,1,7
2,6,3,13,2013,0,2,13,2013,1,1,100,4,7
3,6,3,13,2013,0,2,13,2013,1,8,62,2,7
4,6,3,13,2013,0,2,13,2013,1,4,52,1,7


In [20]:
from sklearn.multiclass import OutputCodeClassifier
from sklearn.ensemble import RandomForestClassifier
X, y = train_data,train_data['Offense']
clf2 = OutputCodeClassifier(RandomForestClassifier(random_state=0),
                           code_size=2, random_state=0)
pred=clf2.fit(X, y).predict(X)

In [409]:
from sklearn.multiclass import OutputCodeClassifier
from sklearn.ensemble import RandomForestClassifier


X, y = data.drop(["Location 1"], axis=1),data['Offense']
clf = OutputCodeClassifier(RandomForestClassifier(random_state=0),
                           code_size=2, random_state=0)
pred=clf.fit(X, y).predict(X)

In [451]:
## DROP COLUMNS AND ENCODE TEST DATA

In [32]:
test=pd.read_csv("/Users/deepak/Downloads/data_science_challenge/test.csv")
test_data=drop_colums(test)
test_data.head(2)

,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Sector,Precinct,Borough,Jurisdiction
0,Monday,Oct,2,2015,12,11,23,2015,GRAND LARCENY,B,25,MANHATTAN,N.Y. POLICE DEPT
1,Wednesday,Sep,6,2015,2,10,2,2015,FELONY ASSAULT,G,90,BROOKLYN,N.Y. HOUSING POLICE


In [33]:
obj_test = test.select_dtypes(include=['object']).copy()
obj_test.head()

,Identifier,Occurrence Datetime,Day of Week,Occurrence Month,Offense,Sector,Borough,Jurisdiction,Location 1,Occurrence Date
0,aae098f0,10/02/2015 12:11:00 PM,Monday,Oct,GRAND LARCENY,B,MANHATTAN,N.Y. POLICE DEPT,"(40.8043840460001, -73.9374216689999)",2015-10-02
1,d71bac4b,09/06/2015 02:00:00 AM,Wednesday,Sep,FELONY ASSAULT,G,BROOKLYN,N.Y. HOUSING POLICE,"(40.703707008, -73.943257966)",2015-09-06
2,ea9926ff,12/10/2015 04:45:00 PM,Sunday,Dec,FELONY ASSAULT,C,MANHATTAN,N.Y. POLICE DEPT,"(40.7613110390001, -73.973083376)",2015-12-10
3,ae713901,12/01/2015 02:30:00 PM,Friday,Dec,GRAND LARCENY,H,BROOKLYN,N.Y. POLICE DEPT,"(40.7175732060001, -73.9625349519999)",2015-12-01
4,ab905411,02/13/2015 12:01:00 AM,Friday,Feb,GRAND LARCENY,B,BROOKLYN,N.Y. POLICE DEPT,"(40.65030209, -73.9495198819999)",2015-02-13


In [42]:
encode(obj_test,test_data)
test_data.head()
test_data=drop_colums(test_data)

In [43]:
from sklearn.metrics import accuracy_score

y_pred=clf2.predict(test_data)
y_true=test_data["Offense"]
accuracy_score(y_true, y_pred)

0.99977136949671463

In [443]:
y_pred=clf.predict(test_data)

In [445]:
from sklearn.metrics import accuracy_score
y_true=test_data["Offense"]
accuracy_score(y_true, y_pred)

0.99573554409089549


1. How would you improve your model if you had another hour?

Lat long is an important feature. Certian type of crimes are possibly related to certain specific areas.I would spend more time on seeing if I can convert them to ZIPCODE via  an online database.

2. How would you improve your model if you had another week?

Spend more time on learning about time series problems,
Use some of the dropped features.
Spend more time visualizing.


3. What approach did you use?
Random Forest


4. Why did you use this approach?
Much Faster than SVM

